In [2]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['대화 요약']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            dialog_id = line['id']+'.1.'
            speaker = line['metadata']['speaker']
            utterance = line['utterance']
            for utt in utterance:
                utt['id'] = utt['id'].replace(dialog_id, '')
            
            main_summary = line['SC']['main_summary']
            topic = line['SC']['topic']
            utt_group_dict = {}
            for sentence in line['discourse']['sentence_set']:
                utt_group_dict[sentence['sentence_set_id']] = [uid.replace(dialog_id,'') 
                                                               for uid in sentence['utterance_id']]
            for topic_dict in topic:
                topic_dict['reference'] = [uid 
                                                     for gid in topic_dict.pop('utterance_group') 
                                                     for uid in utt_group_dict[gid['sentence_set_id']]]
            #### data preprocess end 
            #### 대화 요약
            data = {'input': {}, 'output': {}}
            ## preprocess data from line
            data['input']['speaker'] = speaker
            data['input']['uttrance'] = utterance
            data['output']['main_summary'] = main_summary
            data['output']['topic_summaries'] = topic
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 대화 요약 end
            
            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

train: 100%|██████████| 1/1 [00:01<00:00,  1.95s/it]
valid: 0it [00:00, ?it/s]
